In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split

In [2]:
#Normalizaition
def Normalization(X_train,X_test):
    Xmin= np.min(X_train,axis=0)
    Xmax =np.max(X_train,axis=0)
    X_train=(X_train - Xmin)/(Xmax - Xmin)
    X_test=(X_test - Xmin)/(Xmax - Xmin)
    return X_train,X_test

### 1. Hãy xây dựng mô hình logistic regression bằng tất cả các features trong file heart, so sánh với thư viện sklearn.


In [3]:
data=pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-03/heart.csv")
data.head()



,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:

X= data.iloc[:,:-1].values
y= data.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=42)
X_train_norm,X_test_norm = Normalization(X_train,X_test)

# y_train=y_train.reshape([-1,1])
# y_test=y_test.reshape([-1,1])


**Logistic Regression**
$N$ là số samples , $m$ là số features

$$
X =
\begin{bmatrix}
1 & x_1^{(1)} & x_2^{(1)}    &\cdots   &x_m^{(1)}    \\
1 & x_1^{(1)} & x_2^{(1)}    &\cdots   &x_m^{(2)}    \\
1 & \cdots    & \cdots       &\cdots   &  \\
1 & x_1^{(N)} & x_2^{(N)}     &\cdots  &x_m^{(N)}    \\
\end{bmatrix} , 
y = \begin{bmatrix}
y_1 \\
y_2 \\
\cdots \\
y_N
\end{bmatrix} ,
W = \begin{bmatrix}
\omega_0 \\
\omega_1 \\
\cdots \\
\omega_m
\end{bmatrix}
$$ 

$ \hat{y} = \sigma(XW) $ 

$ J = -\dfrac{1}{N}\sum\limits_{i=1}^{N}\Big[y_i  log(\hat{y_i}) + (1-y_i) log(1 - \hat{y_i})\Big]$

$\dfrac{dJ}{dW} = X^T*(\hat{y} - y)$

Cập nhật $W$ : 
$$ W:= W - \alpha *  \dfrac{dJ}{dW}$$

In [5]:

def sigmoid (x) :
    '''
    Tính hàm sigmoid (đầu ra dự đoán của bài toán Logistic Regression)
    '''
    return 1/(1+ np.exp(-x))
    

class Logistic_Regression :
    '''
    Logistic Regression là 1 thuật toán phân loại được dùng để gán các 
    đối tượng cho 1 tập hợp giá trị rời rạc (như 0, 1, 2, ...).
    '''
    def __init__(self,n_epoch , lr):
        '''
        Khởi tạo các tham số cho mô hình
        Tham số :
        n_epoch -- Số epoch mà ta muốn chạy
        lr -- tốc độ học của thuật toán  
        '''
        self.n_epoch = n_epoch
        self.lr = lr
    
    def fit(self,X_train,y_train) :
        '''
        Học W tốt nhất
        Tham số :
        X_train -- dữ liệu tập huấn luyện
        y_train -- label của tập huấn luyện
        '''
        np.random.seed(5)
        y_train = y_train.reshape(-1,1)
        X_concat = np.concatenate([np.ones([X_train.shape[0],1]), X_train],axis = 1)
        self.N = X_train.shape[0]
        W = np.random.normal(0,1, size = (X_concat.shape[1],1))

        for epoch in range(self.n_epoch) :
            y_hat = sigmoid(np.dot(X_concat,W))
            loss = -(1/self.N) *  np.sum(y_train * np.log(y_hat) + (1-y_train)*np.log(1-y_hat))

            gradient = (1/self.N) * np.dot(X_concat.T, y_hat - y_train)
            W = W - self.lr * gradient
            if epoch % 1000 == 0 :
                print('Epoch : {} ---------------- Loss : {}'.format(epoch,loss))
        self.W = W
                
    def predict(self,X_test):
        '''
        Dự đoán label của tập kiểm tra từ W đã được học
        Tham số : 
        X_test -- tập kiểm tra
        Trả về : 
        Label của tập kiểm tra 
        '''
        ones = np.ones((X_test.shape[0],1 ))
        self.X_test_concat = np.concatenate((ones,X_test), axis = 1)
        y_hat = sigmoid(np.dot(self.X_test_concat,self.W))
        y_predict = np.where(y_hat > 0.5, 1,0)
        return y_predict
    
    def score(self,X_test,y_test):
        '''
        Tính score giữa label dự đoán và label thật
        Tham số :
        X_test -- tập kiểm tra (mô hình dự đoán label từ tập này)
        y_test -- label thật 
        Trả về :
        Số trong khoảng [0,1] nói về khả năng học của mô hình.
        '''
        y_test = y_test.reshape(-1,1)
        ones = np.ones((X_test.shape[0],1 ))
        self.X_test_concat = np.concatenate((ones,X_test), axis = 1)
        y_hat = sigmoid(np.dot(self.X_test_concat,self.W))
        y_predict = np.where(y_hat > 0.5, 1,0)
        return np.mean(y_predict  == y_test)
        

In [6]:

model = Logistic_Regression( n_epoch = 50000 , lr = 0.1)
model.fit(X_train_norm,y_train)


Epoch : 0 ---------------- Loss : 1.285659017534689
Epoch : 1000 ---------------- Loss : 0.34669271100798593
Epoch : 2000 ---------------- Loss : 0.32535421610966303
Epoch : 3000 ---------------- Loss : 0.3172479575630845
Epoch : 4000 ---------------- Loss : 0.31315146825981977
Epoch : 5000 ---------------- Loss : 0.3108319603053405
Epoch : 6000 ---------------- Loss : 0.30942985649138305
Epoch : 7000 ---------------- Loss : 0.30854322474929385
Epoch : 8000 ---------------- Loss : 0.30796291879976656
Epoch : 9000 ---------------- Loss : 0.30757235280743306
Epoch : 10000 ---------------- Loss : 0.3073032482939276
Epoch : 11000 ---------------- Loss : 0.30711406072652736
Epoch : 12000 ---------------- Loss : 0.3069787108659649
Epoch : 13000 ---------------- Loss : 0.3068803881859713
Epoch : 14000 ---------------- Loss : 0.3068080031348633
Epoch : 15000 ---------------- Loss : 0.3067540878683341
Epoch : 16000 ---------------- Loss : 0.30671351902911875
Epoch : 17000 ---------------- Loss 

In [7]:

print('Kết quả chạy bằng mô hình : ',model.score(X_test_norm,y_test))


Kết quả chạy bằng mô hình :  0.81


In [8]:

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(
    random_state=1000,
    penalty= 'none'
).fit(X_train_norm, y_train)

print('Kết quả chạy bằng thư viện : ',clf.score(X_test_norm,y_test))

Kết quả chạy bằng thư viện :  0.81


### 2. Hãy xây dựng mô hình softmax regression trên bộ Iris (nên Normalize data), so sánh với thư viện sklearn.

In [9]:

from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train_norm,X_test_norm  = Normalization(X_train,X_test)


**Softmax Regression** : 

Đầu ra của bài toán Softmax khác với bài toán Logistic Regression ở hàm kích hoạt :

$\hat{y} = softmax(XW)$

Loss : 

$L = -\sum\limits_{i=1}^{c}y_i log(\hat{y_i})$ ($c$ là số class của bài toán).

In [10]:

def softmax(z):
    return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T


class Multiclass_LogisticRegression:
    '''
    Softmax Regression khác với bài toán Logistic Regression ở chỗ đầu ra sẽ có nhiều lớp hơn.
    '''
    def __init__(self, n_epoch, lr ) :
        '''
        Khởi tạo các tham số cho mô hình
        Tham số :
        n_epoch -- Số epoch mà ta muốn chạy
        lr -- tốc độ học của thuật toán  
        '''
        self.n_epoch = n_epoch
        self.lr = lr
    
    def fit(self, X_train,y_train):
        '''
        Chọn mô hình học W tốt nhất
        Tham số :
        X_train -- dữ liệu tập huấn luyện
        y_train -- label của tập huấn luyện
        '''
        np.random.seed(5)
        self.N = X_train.shape[0]
        X_concat = np.concatenate([np.ones([self.N,1]), X_train],axis = 1)
        W = np.random.normal(0,1, size = (X_concat.shape[1],y_train.max() + 1 ))
        
        y_train_onehot = np.zeros( (y_train.size, y_train.max() + 1),dtype=int)
        y_train_onehot[np.arange(y_train.size), y_train.reshape(-1)] = 1
        self.archived_W =  [W]   # Tạo ra list để lưu W
        self.archived_loss = [10]  # tạo ra list để luu loss

        for epochs in range(self.n_epoch):
            y_hat = softmax(np.dot(X_concat,W))
            loss = -(1/self.N) *  np.sum(y_train_onehot * np.log(y_hat))
            self.archived_loss.append(loss)
            gradient = (1/self.N) * np.dot(X_concat.T, y_hat - y_train_onehot)
            W = W - self.lr * gradient
            self.archived_W.append(W)
            if epochs % 1000 == 0 :
                print('Epoch {}  -----------  Loss {}'.format(epochs,loss))
            self.W = W
            
    def predict(self,X_test) :
        '''
        Dự đoán label của tập kiểm tra từ W đã được học
        Tham số : 
        X_test -- tập kiểm tra
        Trả về : 
        Label của tập kiểm tra 
        '''
        X_test_concat = np.concatenate([np.ones([X_test.shape[0],1]),X_test],axis=1)
        y_hat = softmax(np.dot(X_test_concat,self.W))
        y_predict = np.argmax(y_hat,axis = 1)
        return y_predict
    
    def score(self,X_test,y_test):
        '''
        Tính score giữa label dự đoán và label thật
        Tham số :
        X_test -- tập kiểm tra (mô hình dự đoán label từ tập này)
        y_test -- label thật 
        Trả về :
        Số trong khoảng [0,1] nói về khả năng học của mô hình.
        '''
        self.X_test_concat = np.concatenate([np.ones([X_test.shape[0],1]),X_test],axis=1)
        y_hat = softmax(np.dot(self.X_test_concat,self.W))
        y_predict = np.argmax(y_hat,axis = 1)
        return np.mean(y_predict == y_test)

In [11]:

model = Multiclass_LogisticRegression(n_epoch = 15000, lr = 0.01 )
model.fit(X_train_norm,y_train)



Epoch 0  -----------  Loss 2.108380248317868
Epoch 1000  -----------  Loss 0.878363238759448
Epoch 2000  -----------  Loss 0.691397045306415
Epoch 3000  -----------  Loss 0.5966185035410634
Epoch 4000  -----------  Loss 0.5369631571275945
Epoch 5000  -----------  Loss 0.49459886201008657
Epoch 6000  -----------  Loss 0.4622576318801885
Epoch 7000  -----------  Loss 0.43637079127035777
Epoch 8000  -----------  Loss 0.41494781727759183
Epoch 9000  -----------  Loss 0.39677398754749404
Epoch 10000  -----------  Loss 0.3810577062652737
Epoch 11000  -----------  Loss 0.36725687634849297
Epoch 12000  -----------  Loss 0.3549856098363435
Epoch 13000  -----------  Loss 0.3439605408014571
Epoch 14000  -----------  Loss 0.3339681874935225


In [12]:
print('Kết quả chạy bằng mô hình : ',model.score(X_test_norm,y_test))


Kết quả chạy bằng mô hình :  0.9


In [13]:


clf = LogisticRegression(random_state=0).fit(X_train_norm, y_train)

print('Kêt quả chạy bằng thư viện :', clf.score(X_test_norm,y_test))

Kêt quả chạy bằng thư viện : 0.9
